# fastai bugs

This is a minimal notebook to illustrate some bugs/inconveniences in the fastai library which I would be happy to fix.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from fastai.imports import *
from fastai.conv_learner import *
from fastai.dataset import *
import numpy as np

## Dimension ordering for ImageClassifierData.from_arrays

Using the ImageClassifierData.from_arrays allows us to create a data object from numpy arrays but there seems to be a mismatch between the .sz attribute and the dimension ordering required by PyTorch.

Let's create an example dataset with 3 channels and size 128x128, a batch size of 64 and 65 training images.

In [3]:
ntrn=65

The ArraysDataset class uses x.shape[1] to set the sz attribute

If we use [nsample, x, y, channel], this would therefore result in sz=128. 

In [56]:
data1 = ImageClassifierData.from_arrays(
    path='./',
    trn=(np.random.rand(ntrn, 128, 128, 3), np.random.randint(0, 1, ntrn)),
    val=(np.random.rand(ntrn, 128, 128, 3), np.random.randint(0, 1, ntrn)),
    bs=64
)

In [57]:
data1.sz

128

The sz attribute in turn is used to create the shapes for the learner objects.

In [73]:
learn1 = ConvLearner.pretrained(resnet18, data1)

In [74]:
learn1.summary()

OrderedDict([('Conv2d-1',
              OrderedDict([('input_shape', [-1, 3, 128, 128]),
                           ('output_shape', [-1, 64, 64, 64]),
                           ('trainable', False),
                           ('nb_params', 9408)])),
             ('BatchNorm2d-2',
              OrderedDict([('input_shape', [-1, 64, 64, 64]),
                           ('output_shape', [-1, 64, 64, 64]),
                           ('trainable', False),
                           ('nb_params', 128)])),
             ('ReLU-3',
              OrderedDict([('input_shape', [-1, 64, 64, 64]),
                           ('output_shape', [-1, 64, 64, 64]),
                           ('nb_params', 0)])),
             ('MaxPool2d-4',
              OrderedDict([('input_shape', [-1, 64, 64, 64]),
                           ('output_shape', [-1, 64, 32, 32]),
                           ('nb_params', 0)])),
             ('Conv2d-5',
              OrderedDict([('input_shape', [-1, 64, 32, 32]),
      

But PyTorch expects [nbatch, channel, x, y], which is why training will throw an error.

In [65]:
learn1.fit(0.1, 1)

A Jupyter Widget

  0%|          | 0/2 [00:00<?, ?it/s]


RuntimeError: Need input.size[1] == 3 but got 128 instead.

Chosing [nsample, x, y, channel] gives us the wrong .sz attribute and therefore the wrong model shape.

In [4]:
data2 = ImageClassifierData.from_arrays(
    path='./',
    trn=(np.random.rand(ntrn, 3, 128, 128), np.random.randint(0, 1, ntrn)),
    val=(np.random.rand(ntrn, 3, 128, 128), np.random.randint(0, 1, ntrn)),
    bs=64
)

In [5]:
data2.sz

128

In [8]:
learn2 = ConvLearner.pretrained(resnet18, data2)

In [9]:
learn2.fit(0.1, 1)

A Jupyter Widget

epoch      trn_loss   val_loss   accuracy            
    0      0.0        nan        0.0       



[nan, 0.0]